# 6章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# 6-1
!mkdir chap6
%cd ./chap6

/content/chap6


In [2]:
# 6-2
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 48.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 60.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 59.5 MB/s eta 0:00:00
  Created wheel for fugashi: filename=fugashi-1.1.0-cp310-cp310-linux_x86_64.whl size=211029 sha256=eefea30e8d50a1895f1c4a121aa0bd4c655c5f5b941ba933db89f7c09dcb97de
  Stored in directory: /root/.cache/pip/wheels/f6/bc/9b/6d38a64c5bea

In [1]:
# 6-3
import random
import glob
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import pytorch_lightning as pl

# 日本語の事前学習モデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [2]:
# 6-4
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
bert_sc = BertForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=2
)
# bert_sc = bert_sc.cuda()
bert_sc = bert_sc.cpu()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# 6-5
text_list = [
    "この映画は面白かった。",
    "この映画の最後にはがっかりさせられた。",
    "この映画を見て幸せな気持ちになった。"
]
label_list = [1,0,1]

# データの符号化
encoding = tokenizer(
    text_list,
    padding = 'longest',
    return_tensors='pt'
)
# encoding = { k: v.cuda() for k, v in encoding.items() }
encoding = { k: v.cpu() for k, v in encoding.items() }

# labels = torch.tensor(label_list).cuda()
labels = torch.tensor(label_list).cpu()

# 推論
with torch.no_grad():
    output = bert_sc.forward(**encoding)
scores = output.logits # 分類スコア
labels_predicted = scores.argmax(-1) # スコアが最も高いラベル
num_correct = (labels_predicted==labels).sum().item() # 正解数
accuracy = num_correct/labels.size(0) # 精度

print("# scores:")
print(scores.size())
print("# predicted labels:")
print(labels_predicted)
print("# accuracy:")
print(accuracy)

# scores:
torch.Size([3, 2])
# predicted labels:
tensor([1, 1, 1])
# accuracy:
0.6666666666666666


In [5]:
# 6-6
# 符号化
encoding = tokenizer(
    text_list,
    padding='longest',
    return_tensors='pt'
)
encoding['labels'] = torch.tensor(label_list) # 入力にラベルを加える。
# encoding = { k: v.cuda() for k, v in encoding.items() }
encoding = { k: v.cpu() for k, v in encoding.items() }

# ロスの計算
output = bert_sc(**encoding)
loss = output.loss # 損失の取得
print(loss)

tensor(0.6875, grad_fn=<NllLossBackward0>)


In [9]:
# 6-7
#データのダウンロード
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
#ファイルの解凍
!tar -zxf ldcc-20140209.tar.gz

'wget' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


tar: Error opening archive: Failed to open 'ldcc-20140209.tar.gz'


In [14]:
# 6-8
!cat ./text/it-life-hack/it-life-hack-6342280.txt # ファイルを表示

'cat' �́A�����R�}���h�܂��͊O���R�}���h�A
����\�ȃv���O�����܂��̓o�b�` �t�@�C���Ƃ��ĔF������Ă��܂���B


In [6]:
with open('.\\text\\it-life-hack\\it-life-hack-6292880.txt', encoding='UTF8') as f:
    file_read = f.read()

file_read

'http://news.livedoor.com/article/detail/6292880/\n2012-02-19T13:00:00+0900\n旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック【ITフラッシュバック】\nテレビやTwitterと連携できるパソコンや、プロセッサや切り替わるパソコンなど、面白いパソコンが次から次へと登場した。旧式Macの禁断ともいえるパワーアップ方法から、NECの最新PC、話題のThinkPad X1 Hybrid、新セキュリティソフトまで一挙に紹介しよう。\n\n■インテル SSD 520をMacに装着！旧式Macはどれほど高速化するのか (上)\nインテルが最新SSD「520シリーズ」を発売した。現行SSDの中でもトップクラスの性能を誇る同製品を、旧型Macの高速化を図るというポイントでレビューしてみた。少し風変わりなレビューとなるが、どの程度の効果があるか、期待大である。\n\n\n■http://itlifehack.jp/archives/6716997.html\nThinkPad X1 Hybridは使用するCPUがx86(インテルCore iなど)からARMに切り替わるハイブリッドなPCだが、これと同時にOSも切り替わる。\n\n\n■初期費用、更新費用ともに無料！ジャストシステム、ヤモリが目印のセキュリティソフト\n現在では、多くのユーザーがパソコンにセキュリティソフトを導入しているが、その過半数は毎年5,000円程度かかる更新費用やその手続きについて不満を持っている。有料ソフトを利用するユーザーの約8割は無料のセキュリティソフトを知っているにもかかわらず、性能面で劣るのではという不安から導入を控えているという状況にある。\n\n\n■テレビの新しい楽しみ方を提案！NECの春PCはTVとTwitterの連携\nNECは2012年2月14日、個人向けデスクトップパソコン「VALUESTAR」シリーズ3タイプ16モデルを2月16日より販売すると発表した。新商品では、よりパワフルになった録画機能に加え、TV視聴・録画機能に業界で初めて人気のTwitterを連携させた「SmartVisionつぶやきプラス」を追加するなど、TVパソコンならではの機能を搭載。スマートフォン、ホームネットワーク対応も強化し

In [7]:
# 6-9
# データローダーの作成
dataset_for_loader = [
    {'data':torch.tensor([0,1]), 'labels':torch.tensor(0)},
    {'data':torch.tensor([2,3]), 'labels':torch.tensor(1)},
    {'data':torch.tensor([4,5]), 'labels':torch.tensor(2)},
    {'data':torch.tensor([6,7]), 'labels':torch.tensor(3)},
]
loader = DataLoader(dataset_for_loader, batch_size=2)

# データセットからミニバッチを取り出す
for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)
    ## ファインチューニングではここでミニバッチ毎の処理を行う

# batch 0
{'data': tensor([[0, 1],
        [2, 3]]), 'labels': tensor([0, 1])}
# batch 1
{'data': tensor([[4, 5],
        [6, 7]]), 'labels': tensor([2, 3])}


In [8]:
# 6-10
loader = DataLoader(dataset_for_loader, batch_size=2, shuffle=True)

for idx, batch in enumerate(loader):
    print(f'# batch {idx}')
    print(batch)

# batch 0
{'data': tensor([[6, 7],
        [2, 3]]), 'labels': tensor([3, 1])}
# batch 1
{'data': tensor([[4, 5],
        [0, 1]]), 'labels': tensor([2, 0])}


In [9]:
# 6-11
# カテゴリーのリスト
category_list = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

# トークナイザのロード
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

# 各データの形式を整える
max_length = 128
dataset_for_loader = []
for label, category in enumerate(tqdm(category_list)):
    for file in glob.glob(f'.\\text\\{category}\\{category}*'):
        lines = open(file, encoding='utf-8').read().splitlines()
        text = '\n'.join(lines[3:]) # ファイルの4行目からを抜き出す。
        encoding = tokenizer(
            text,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )
        encoding['labels'] = label # ラベルを追加
        encoding = { k: torch.tensor(v) for k, v in encoding.items() }
        dataset_for_loader.append(encoding)

100%|██████████| 9/9 [00:27<00:00,  3.06s/it]


In [10]:
# 6-12
print(dataset_for_loader[0])

{'input_ids': tensor([    2,  2340, 19693, 10585, 28459,    35,  6692, 28493,    13,   501,
           62,   101,    37,     8,   569,   335,     5,    51,     7,     9,
         1040,     5,   616,     9,  2941,    18,  5602,   501,    20,    16,
         4027, 10531,   140,    36,    73, 30020, 28457, 25127,    38,  1080,
            5,    53,    28,   707,     5,    12,     9,    80,  3635,   205,
           29,  2935,   604,  5846,  6503,    11,  4722,    16,   861,    13,
            6, 12272, 24050,  2079,    11,    26,    62,    45,    28,  2451,
           80,     8,    36, 24050,    14,    31,  1058,    75, 11218, 10531,
         3676,   542,     5, 22130,     6,  5408,    16,  4831,    80,    29,
           18,  7045,    26, 28456,  4799,   900,     6,   569,   335,     9,
         1704,  1277,    15,  3318,  2575,    29,  2935,  5233,    75,    13,
         3472,   459,    12,  8585,  3171,   312,  3676,   542, 22130,   241,
            5,   709, 28696,  2180,    14, 12959, 

In [11]:
# 6-13
# データセットの分割
random.shuffle(dataset_for_loader) # ランダムにシャッフル
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train] # 学習データ
dataset_val = dataset_for_loader[n_train:n_train+n_val] # 検証データ
dataset_test = dataset_for_loader[n_train+n_val:] # テストデータ

# データセットからデータローダを作成
# 学習データはshuffle=Trueにする。
dataloader_train = DataLoader(
    dataset_train, batch_size=32, shuffle=True
)
dataloader_val = DataLoader(dataset_val, batch_size=256)
dataloader_test = DataLoader(dataset_test, batch_size=256)

In [13]:
# 6-14
class BertForSequenceClassification_pl(pl.LightningModule):

    def __init__(self, model_name, num_labels, lr):
        # model_name: Transformersのモデルの名前
        # num_labels: ラベルの数
        # lr: 学習率

        super().__init__()

        # 引数のnum_labelsとlrを保存。
        # 例えば、self.hparams.lrでlrにアクセスできる。
        # チェックポイント作成時にも自動で保存される。
        self.save_hyperparameters()

        # BERTのロード
        self.bert_sc = BertForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

    # 学習データのミニバッチ(`batch`)が与えられた時に損失を出力する関数を書く。
    # batch_idxはミニバッチの番号であるが今回は使わない。
    def training_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        loss = output.loss
        self.log('train_loss', loss) # 損失を'train_loss'の名前でログをとる。
        return loss

    # 検証データのミニバッチが与えられた時に、
    # 検証データを評価する指標を計算する関数を書く。
    def validation_step(self, batch, batch_idx):
        output = self.bert_sc(**batch)
        val_loss = output.loss
        self.log('val_loss', val_loss) # 損失を'val_loss'の名前でログをとる。

    # テストデータのミニバッチが与えられた時に、
    # テストデータを評価する指標を計算する関数を書く。
    def test_step(self, batch, batch_idx):
        labels = batch.pop('labels') # バッチからラベルを取得
        output = self.bert_sc(**batch)
        labels_predicted = output.logits.argmax(-1)
        num_correct = ( labels_predicted == labels ).sum().item()
        accuracy = num_correct/labels.size(0) #精度
        self.log('accuracy', accuracy) # 精度を'accuracy'の名前でログをとる。

    # 学習に用いるオプティマイザを返す関数を書く。
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [15]:
# 6-15
# 学習時にモデルの重みを保存する条件を指定
checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='model/',
)

# 学習の方法を指定
trainer = pl.Trainer(
    # gpus=1,
    max_epochs=10,
    callbacks = [checkpoint]
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [16]:
# 6-16
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl(
    MODEL_NAME, num_labels=9, lr=1e-5
)

# ファインチューニングを行う。
trainer.fit(model, dataloader_train, dataloader_val)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Missing logger folder: d:\GitHub\test\lightning_logs

  | Name    | Type                          | Params
----------------------------------------------------------
0 | bert_sc | BertForSequenceClassification | 110 M 
----------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.497   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\lucky\AppData\Local\Programs\Python\Python38\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\lucky\AppData\Local\Programs\Python\Python38\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [17]:
# 6-17
best_model_path = checkpoint.best_model_path # ベストモデルのファイル
print('ベストモデルのファイル: ', checkpoint.best_model_path)
print('ベストモデルの検証データに対する損失: ', checkpoint.best_model_score)

ベストモデルのファイル:  D:\GitHub\test\model\epoch=3-step=556.ckpt
ベストモデルの検証データに対する損失:  tensor(0.3939)


In [18]:
# 6-18
%load_ext tensorboard
%tensorboard --logdir ./

In [19]:
# 6-19
test = trainer.test(dataloaders=dataloader_test)
print(f'Accuracy: {test[0]["accuracy"]:.2f}')

c:\Users\lucky\AppData\Local\Programs\Python\Python38\lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:149: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at D:\GitHub\test\model\epoch=3-step=556.ckpt


Loaded model weights from the checkpoint at D:\GitHub\test\model\epoch=3-step=556.ckpt
c:\Users\lucky\AppData\Local\Programs\Python\Python38\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:438: PossibleUserWarning: The dataloader, test_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         accuracy          │     0.885345995426178     │
└───────────────────────────┴───────────────────────────┘

Accuracy: 0.89


In [20]:
# 6-20
# PyTorch Lightningモデルのロード
model = BertForSequenceClassification_pl.load_from_checkpoint(
    best_model_path
)

# Transformers対応のモデルを./model_transformesに保存
model.bert_sc.save_pretrained('./model_transformers')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# 6-21
bert_sc = BertForSequenceClassification.from_pretrained(
    './model_transformers'
)